In [3]:
import speech_recognition as sr
from gtts import gTTS
from playsound import playsound
import os

def speak(text):
    """Converts text to speech and plays the audio."""
    try:
        tts = gTTS(text=text, lang='en')
        # Save the audio file temporarily
        filename = "feedback.mp3"
        tts.save(filename)
        playsound(filename)
        os.remove(filename)
    except Exception as e:
        print(f"Error in text-to-speech: {e}")

def listen_and_recognize(engine='google'):
    """
    Listens for a command from the microphone and uses the specified engine
    to convert it to text.
    """
    # Initialize the recognizer
    r = sr.Recognizer()

    with sr.Microphone() as source:
        # Adjust for ambient noise to improve accuracy
        print("Calibrating for ambient noise, please wait...")
        r.adjust_for_ambient_noise(source, duration=1)
        
        print("\nListening for your command...")
        speak("I'm listening for your command.")
        
        try:
            # Listen for audio input from the user
            audio = r.listen(source, timeout=5, phrase_time_limit=5)
            print("Processing your speech...")
            
            # --- Recognition Logic ---
            text = ""
            if engine == 'google':
                # Use Google Web Speech API
                text = r.recognize_google(audio)
            elif engine == 'sphinx':
                # Use CMU Sphinx (offline)
                # Note: Requires installing pocketsphinx (pip install pocketsphinx)
                try:
                    text = r.recognize_sphinx(audio)
                except ImportError:
                    print("Sphinx is not installed. Please run 'pip install pocketsphinx'.")
                    return None
            else:
                print(f"Error: Recognition engine '{engine}' not supported.")
                return None
                
            print(f"You said ({engine}): {text}")
            speak(f"You said: {text}")
            return text

        # --- Error Handling ---
        except sr.WaitTimeoutError:
            print("Error: No speech detected within the time limit.")
            speak("I didn't hear anything. Please try again.")
            return None
        except sr.UnknownValueError:
            print(f"Error: {engine.capitalize()} Speech Recognition could not understand audio.")
            speak("Sorry, I could not understand what you said.")
            return None
        except sr.RequestError as e:
            print(f"Error: Could not request results from {engine.capitalize()} service; {e}")
            speak("My connection to the speech service failed.")
            return None
        except Exception as e:
            print(f"An unexpected error occurred: {e}")
            speak("An unexpected error occurred.")
            return None

# --- Main Execution Block ---
if __name__ == "__main__":
    # Allows comparison between different recognition methods
    print("Welcome to the Real-Time Speech-to-Text System!")
    
    # --- Example 1: Using Google's Engine ---
    print("\n--- Testing with Google Web Speech API ---")
    command_google = listen_and_recognize(engine='google')
    if command_google:
        print(f"Command received (Google): '{command_google}'")

    # --- Example 2: Using Sphinx (Offline) Engine ---
    # Note: Sphinx is less accurate but works offline.
    print("\n--- Testing with CMU Sphinx (Offline) ---")
    command_sphinx = listen_and_recognize(engine='sphinx')
    if command_sphinx:
        print(f"Command received (Sphinx): '{command_sphinx}'")

Welcome to the Real-Time Speech-to-Text System!

--- Testing with Google Web Speech API ---
Calibrating for ambient noise, please wait...

Listening for your command...
Processing your speech...
You said (google): hello
Command received (Google): 'hello'

--- Testing with CMU Sphinx (Offline) ---
Calibrating for ambient noise, please wait...

Listening for your command...
Processing your speech...
You said (sphinx): and you'll
Command received (Sphinx): 'and you'll'
